In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import main
import psycopg2
import re

In [3]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

In [4]:
conn = psycopg2.connect(
    dbname="cs2_database",
    user="antonovdv",
    password="Fktrcfylhjdf1",
    host="localhost"
)

In [5]:
cursor = conn.cursor()
cursor.execute("SELECT * FROM cs2.teams")
cursor.fetchall()

[]

In [6]:
pd.set_option('display.max_rows', None)

In [7]:
top = main.topteams()

In [8]:
teams = pd.DataFrame(top)

In [9]:
teams.head()

,name,rank,rank-points,team-id,team-url,stats-url,team-players
0,Vitality,1,950,NaN,https://hltv.org/team/9565/Vitality,https://www.hltv.org/ranking/teams/2025/novemb...,"[{'name': 'Dan 'apEX' Madesclaire', 'player-id..."
1,FURIA,2,665,NaN,https://hltv.org/team/8297/FURIA,https://www.hltv.org/ranking/teams/2025/novemb...,"[{'name': 'Gabriel 'FalleN' Toledo', 'player-i..."
2,Falcons,3,626,NaN,https://hltv.org/team/11283/Falcons,https://www.hltv.org/ranking/teams/2025/novemb...,"[{'name': 'Nikola 'NiKo' Kovač', 'player-id': ..."
3,The MongolZ,4,603,NaN,https://hltv.org/team/6248/The MongolZ,https://www.hltv.org/ranking/teams/2025/novemb...,"[{'name': 'Garidmagnai 'bLitz' Byambasuren', '..."
4,MOUZ,5,592,NaN,https://hltv.org/team/4494/MOUZ,https://www.hltv.org/ranking/teams/2025/novemb...,"[{'name': 'Ludvig 'Brollan' Brolin', 'player-i..."


In [10]:
idna_mask = teams["team-id"].isna()

In [11]:
teams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261 entries, 0 to 260
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          261 non-null    object 
 1   rank          261 non-null    int64  
 2   rank-points   261 non-null    int64  
 3   team-id       199 non-null    float64
 4   team-url      261 non-null    object 
 5   stats-url     261 non-null    object 
 6   team-players  261 non-null    object 
dtypes: float64(1), int64(2), object(4)
memory usage: 14.4+ KB


In [12]:
teams.loc[idna_mask, :]["name"]

0                Vitality
1                   FURIA
2                 Falcons
3             The MongolZ
4                    MOUZ
5                  Spirit
6                  Aurora
7           Natus Vincere
8                      G2
9                   3DMAX
10                   FaZe
11                 Liquid
12                 Legacy
13               Astralis
14                   paiN
15            GamerLegion
16                 HEROIC
17           Gentle Mates
18                     B8
19                    SAW
20                  TYLOO
21             Virtus.pro
22                 fnatic
23                    M80
24                   HOTU
25           Inner Circle
26      Ninjas in Pyjamas
27                    FUT
28                BetBoom
29             Passion UA
30            Lynn Vision
31                   9INE
32             PARIVISION
33               FlyQuest
34                   MIBR
35                     9z
36                   ENCE
37                    ESC
38          

In [13]:
for name in teams.loc[idna_mask, :]["name"]:
    teams.loc[teams["name"] == name, "team-id"] = float(teams.loc[teams["name"] == name, "team-url"].item().split("/")[-2])

In [14]:
teams.head()

,name,rank,rank-points,team-id,team-url,stats-url,team-players
0,Vitality,1,950,9565.0,https://hltv.org/team/9565/Vitality,https://www.hltv.org/ranking/teams/2025/novemb...,"[{'name': 'Dan 'apEX' Madesclaire', 'player-id..."
1,FURIA,2,665,8297.0,https://hltv.org/team/8297/FURIA,https://www.hltv.org/ranking/teams/2025/novemb...,"[{'name': 'Gabriel 'FalleN' Toledo', 'player-i..."
2,Falcons,3,626,11283.0,https://hltv.org/team/11283/Falcons,https://www.hltv.org/ranking/teams/2025/novemb...,"[{'name': 'Nikola 'NiKo' Kovač', 'player-id': ..."
3,The MongolZ,4,603,6248.0,https://hltv.org/team/6248/The MongolZ,https://www.hltv.org/ranking/teams/2025/novemb...,"[{'name': 'Garidmagnai 'bLitz' Byambasuren', '..."
4,MOUZ,5,592,4494.0,https://hltv.org/team/4494/MOUZ,https://www.hltv.org/ranking/teams/2025/novemb...,"[{'name': 'Ludvig 'Brollan' Brolin', 'player-i..."


In [15]:
teams.to_csv("teams.csv", index=False)

In [16]:
from bs4 import BeautifulSoup
import requests
import zoneinfo
import tzlocal

HLTV_COOKIE_TIMEZONE = "Europe/Copenhagen"
HLTV_ZONEINFO=zoneinfo.ZoneInfo(HLTV_COOKIE_TIMEZONE)
LOCAL_TIMEZONE_NAME = tzlocal.get_localzone_name()
LOCAL_ZONEINFO = zoneinfo.ZoneInfo(LOCAL_TIMEZONE_NAME)

headers = {
        "referer": "https://www.hltv.org",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    }

cookies = {
        "hltvTimeZone": HLTV_COOKIE_TIMEZONE
    }

url = "https://www.hltv.org/player/21167/donk"

page = requests.get(url, headers=headers, cookies=cookies)

print(page.status_code)

200


In [17]:
url = "https://www.hltv.org/player/21167/donk"

In [18]:
url = "https://www.hltv.org/stats/players/21167/donk"

In [19]:
from datetime import datetime

In [20]:
from dateutil.relativedelta import relativedelta

In [21]:
def get_n_months_before(today_date, n_months):
    return today_date - relativedelta(months=n_months)

In [22]:
today_date = datetime.now().date()
date_3months = get_n_months_before(today_date, 3)
date_6months = get_n_months_before(today_date, 6)
date_7months = get_n_months_before(today_date, 7)

In [23]:
teams.head(5)

,name,rank,rank-points,team-id,team-url,stats-url,team-players
0,Vitality,1,950,9565.0,https://hltv.org/team/9565/Vitality,https://www.hltv.org/ranking/teams/2025/novemb...,"[{'name': 'Dan 'apEX' Madesclaire', 'player-id..."
1,FURIA,2,665,8297.0,https://hltv.org/team/8297/FURIA,https://www.hltv.org/ranking/teams/2025/novemb...,"[{'name': 'Gabriel 'FalleN' Toledo', 'player-i..."
2,Falcons,3,626,11283.0,https://hltv.org/team/11283/Falcons,https://www.hltv.org/ranking/teams/2025/novemb...,"[{'name': 'Nikola 'NiKo' Kovač', 'player-id': ..."
3,The MongolZ,4,603,6248.0,https://hltv.org/team/6248/The MongolZ,https://www.hltv.org/ranking/teams/2025/novemb...,"[{'name': 'Garidmagnai 'bLitz' Byambasuren', '..."
4,MOUZ,5,592,4494.0,https://hltv.org/team/4494/MOUZ,https://www.hltv.org/ranking/teams/2025/novemb...,"[{'name': 'Ludvig 'Brollan' Brolin', 'player-i..."


In [24]:
team = teams.loc[teams["rank"]==11, "name"]

In [25]:
team.values[0]

'FaZe'

In [26]:
pd.DataFrame(teams.loc[teams["name"] == team.values[0], "team-players"].values[0])

,name,player-id,url
0,Finn 'karrigan' Andersen,429,https://www.hltv.org/player/429/karrigan
1,David 'frozen' Čerňanský,9960,https://www.hltv.org/player/9960/frozen
2,Russel 'Twistzz' Van Dulken,10394,https://www.hltv.org/player/10394/twistzz
3,Helvijs 'broky' Saukants,18053,https://www.hltv.org/player/18053/broky
4,Jakub 'jcobbb' Pietruszewski,22383,https://www.hltv.org/player/22383/jcobbb


In [27]:
player = teams.loc[teams["name"] == "FaZe", "team-players"].values[0][2]

In [28]:
player

{'name': "Russel 'Twistzz' Van Dulken",
 'player-id': 10394,
 'url': 'https://www.hltv.org/player/10394/twistzz'}

In [29]:
name = player["name"]
nickname = player["name"].split("'")[1]
id = player["player-id"]

In [30]:
id

10394

In [31]:
def make_types(lib_columns, df):
    df = df.copy()
    df = df.replace('-', '0.0')
    for key in lib_columns.keys():
        if lib_columns[key] == np.datetime64:
            df[key] = pd.to_datetime(df[key])
        elif lib_columns[key] == np.timedelta64:
            df[key] = pd.to_timedelta(df[key])
        else:
            try:
                df[key] = df[key].astype(lib_columns[key])
            except:
                df[key] = df[key].str.split("%")[0][0]
                df[key] = df[key].astype(lib_columns[key])
    return df

In [32]:
def extract_string_between_chars(text, char_start, char_end):
    """
    Извлекает подстроку между двумя символами в строке.

    Args:
        text: Исходная строка.
        char_start: Начальный символ.
        char_end: Конечный символ.

    Returns:
        Подстрока между символами, или None, если один из символов не найден.
    """
    try:
        start = text.index(char_start) + 1
        end = text.index(char_end, start)
        return text[start:end]
    except ValueError:
        return None

In [33]:
start_date3 = str(date_3months)
start_date6 = str(date_6months)
start_date7 = str(date_7months)
end_date = str(today_date)

months3_url = f"https://www.hltv.org/stats/players/{id}/{nickname}?startDate={start_date3}&endDate={end_date}"
months6_url = f"https://www.hltv.org/stats/players/{id}/{nickname}?startDate={start_date6}&endDate={end_date}"
months7_url = f"https://www.hltv.org/stats/players/{id}/{nickname}?startDate={start_date7}&endDate={end_date}"

In [34]:
months3_url

'https://www.hltv.org/stats/players/10394/Twistzz?startDate=2025-08-04&endDate=2025-11-04'

In [35]:
months6_url

'https://www.hltv.org/stats/players/10394/Twistzz?startDate=2025-05-04&endDate=2025-11-04'

In [36]:
months7_url

'https://www.hltv.org/stats/players/10394/Twistzz?startDate=2025-04-04&endDate=2025-11-04'

In [37]:
url = "https://www.hltv.org/stats/players"

In [38]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "none",
    "Sec-Fetch-User": "?1",
    "Cache-Control": "max-age=0",
    "Referer": "https://www.hltv.org/"
}

cookies = {
        "hltvTimeZone": HLTV_COOKIE_TIMEZONE
    }

In [39]:
page.encoding = 'utf-8' 
page = requests.get(months3_url, headers=headers, cookies=cookies)

In [40]:
page.status_code

200

In [41]:
soup = BeautifulSoup(page.text, "lxml")

In [42]:
box_stats = soup.find_all("div", {"class": "role-stats-container standard-box"})
role_stats = soup.find_all("div", {"class": "role-stats-section-columns"})

In [43]:
section = box_stats[0].find("div", {"class": "role-stats-margin role-stats-section-columns"})

In [44]:
xxx = soup.find_all("div", {"class": "role-stats-section role-firepower"})

In [45]:
firepower = xxx[0].find("div", {"class": "role-stats-margin role-stats-section-columns"})

In [46]:
title_firepower = firepower.find_all("div", {"class": "role-stats-title"})

In [47]:
data = firepower.find_all("div", {"class": "role-stats-data"})
extract_string_between_chars(str(data[0]), ">", "<")

'0.77'

In [48]:
data_column = np.zeros((1, len(data)+3))
data_column = data_column.astype(str)
for i in range(len(data)):
    data_column[0, i+3] = extract_string_between_chars(str(data[i]), ">", "<")
data_column[0, 0] = str(id)
data_column[0, 1] = date_3months
data_column[0, 2] = end_date

In [49]:
columns = ["ID",
           "EFFECTIVE_FROM",
           "EFFECTIVE_TO",
           "KPR_both",
            "KPR_CT",
            "KPR_T",
            "KPR_win_both",
            "KPR_win_CT",
            "KPR_win_T",
            "Rounds_with_kills_both",
            "Rounds_with_kills_CT",
            "Rounds_with_kills_T",
            "Rating_both",
            "Rating_CT",
            "Rating_T",
            "DPR_both",
            "DPR_CT",
            "DPR_T",
            "Mulitkills_rounds_both",
            "Mulitkills_rounds_CT",
            "Mulitkills_rounds_T",
            "DPR_win_both",
            "DPR_win_CT",
            "DPR_win_T",
            "Pistol_rating_both",
            "Pistol_rating_CT",
            "Pistol_rating_T",]

In [50]:
result = pd.DataFrame(columns=columns, data=data_column)

In [51]:
percent_columns = [
            "KPR_win_both",
            "KPR_win_CT",
            "KPR_win_T",
            "Rounds_with_kills_both",
            "Rounds_with_kills_CT",
            "Rounds_with_kills_T",
            "Mulitkills_rounds_both",
            "Mulitkills_rounds_CT",
            "Mulitkills_rounds_T",
]

In [52]:
for column in percent_columns:
    result[column] = result[column].str.split("%")[0][0]

In [53]:
columns_types = {
    "ID": np.int32,
    "EFFECTIVE_FROM": np.datetime64,
    "EFFECTIVE_TO": np.datetime64,
    "KPR_both": np.float32,
    "KPR_CT": np.float32,
    "KPR_T": np.float32,
    "KPR_win_both": np.float32,
    "KPR_win_CT": np.float32,
    "KPR_win_T": np.float32,
    "Rounds_with_kills_both": np.float32,
    "Rounds_with_kills_CT": np.float32,
    "Rounds_with_kills_T": np.float32,
    "Rating_both": np.float32,
    "Rating_CT": np.float32,
    "Rating_T": np.float32,
    "DPR_both": np.float32,
    "DPR_CT": np.float32,
    "DPR_T": np.float32,
    "Mulitkills_rounds_both": np.float32,
    "Mulitkills_rounds_CT": np.float32,
    "Mulitkills_rounds_T": np.float32,
    "DPR_win_both": np.float32,
    "DPR_win_CT": np.float32,
    "DPR_win_T": np.float32,
    "Pistol_rating_both": np.float32,
    "Pistol_rating_CT": np.float32,
    "Pistol_rating_T": np.float32,
}

In [54]:
result = make_types(columns_types, result)

In [55]:
result

,ID,EFFECTIVE_FROM,EFFECTIVE_TO,KPR_both,KPR_CT,KPR_T,KPR_win_both,KPR_win_CT,KPR_win_T,Rounds_with_kills_both,Rounds_with_kills_CT,Rounds_with_kills_T,Rating_both,Rating_CT,Rating_T,DPR_both,DPR_CT,DPR_T,Mulitkills_rounds_both,Mulitkills_rounds_CT,Mulitkills_rounds_T,DPR_win_both,DPR_win_CT,DPR_win_T,Pistol_rating_both,Pistol_rating_CT,Pistol_rating_T
0,10394,2025-08-04,2025-11-04,0.77,0.86,0.68,50.0,55.400002,44.5,1.07,1.16,0.97,1.24,1.3,1.17,84.400002,94.099998,74.699997,20.6,22.6,18.6,112.800003,122.5,102.0,1.24,1.35,1.13


In [56]:
def make_dataframe(soup, name: str) -> pd.DataFrame:
    section = soup.find_all("div", {"class": f"role-stats-section role-{name}"})
    stats = section[0].find("div", {"class": "role-stats-margin"})
    data = stats.find_all("div", {"class": "role-stats-data"})
    title_stats = stats.find_all("div", {"class": "role-stats-title"})

    cols = []
    cols.append("ID")
    cols.append("EFFECTIVE_FROM")
    cols.append("EFFECTIVE_TO")

    data_column = np.zeros((1, len(data)+3))
    data_column = data_column.astype(str)
    for i in range(len(data)):
        data_column[0, i+3] = data[i].text
        if (i % 3) == 0:
            cols.append(title_stats[i].text.replace(' ', '_') + "_both")
        elif (i % 3) == 1:
            cols.append(title_stats[i].text.replace(' ', '_') + "_CT")
        elif (i % 3) == 2:
            cols.append(title_stats[i].text.replace(' ', '_') + "_T")
    data_column[0, 0] = str(id)
    data_column[0, 1] = date_3months
    data_column[0, 2] = end_date

    return pd.DataFrame(columns=cols, data=data_column)

## Entrying

In [54]:
xxx1 = soup.find_all("div", {"class": "role-stats-section role-entrying"})

In [55]:
entrying = xxx1[0].find("div", {"class": "role-stats-margin"})

In [56]:
columns = [
    "ID",
    "EFFECTIVE_FROM",
    "EFFECTIVE_TO",
    "Saved_by_teammate_per_round_both",
    "Saved_by_teammate_per_round_CT",
    "Saved_by_teammate_per_round_T",
    "Traded_deaths_per_round_both",
    "Traded_deaths_per_round_CT",
    "Traded_deaths_per_round_T",
    "Traded_deaths_percentage_both",
    "Traded_deaths_percentage_CT",
    "Traded_deaths_percentage_T",
    "Opening_deaths_traded_percentage_both",
    "Opening_deaths_traded_percentage_CT",
    "Opening_deaths_traded_percentage_T",
    "Assists_per_round_both",
    "Assists_per_round_CT",
    "Assists_per_round_T",
    "Support_rounds_both",
    "Support_rounds_CT",
    "Support_rounds_T",
    ]

In [57]:
data = entrying.find_all("div", {"class": "role-stats-data"})

In [58]:
data_column = np.zeros((1, len(data)+3))
data_column = data_column.astype(str)
for i in range(len(data)):
    data_column[0, i+3] = extract_string_between_chars(str(data[i]), ">", "<")
data_column[0, 0] = str(id)
data_column[0, 1] = date_3months
data_column[0, 2] = end_date

In [59]:
result_entrying = pd.DataFrame(columns=columns, data=data_column)

In [60]:
percent_columns = [
            "Traded_deaths_percentage_both",
            "Traded_deaths_percentage_CT",
            "Traded_deaths_percentage_T",
            "Opening_deaths_traded_percentage_both",
            "Opening_deaths_traded_percentage_CT",
            "Opening_deaths_traded_percentage_T",
            "Support_rounds_both",
            "Support_rounds_CT",
            "Support_rounds_T",
]

In [61]:
for column in percent_columns:
    result_entrying[column] = result_entrying[column].str.split("%")[0][0]

In [62]:
columns_types = {
    "ID": np.int32,
    "EFFECTIVE_FROM": np.datetime64,
    "EFFECTIVE_TO": np.datetime64,
    "Saved_by_teammate_per_round_both": np.float32,
    "Saved_by_teammate_per_round_CT": np.float32,
    "Saved_by_teammate_per_round_T": np.float32,
    "Traded_deaths_per_round_both": np.float32,
    "Traded_deaths_per_round_CT": np.float32,
    "Traded_deaths_per_round_T": np.float32,
    "Traded_deaths_percentage_both": np.float32,
    "Traded_deaths_percentage_CT": np.float32,
    "Traded_deaths_percentage_T": np.float32,
    "Opening_deaths_traded_percentage_both": np.float32,
    "Opening_deaths_traded_percentage_CT": np.float32,
    "Opening_deaths_traded_percentage_T": np.float32,
    "Assists_per_round_both": np.float32,
    "Assists_per_round_CT": np.float32,
    "Assists_per_round_T": np.float32,
    "Support_rounds_both": np.float32,
    "Support_rounds_CT": np.float32,
    "Support_rounds_T": np.float32,
}

In [63]:
result_entrying = make_types(columns_types, result_entrying)

In [64]:
result_entrying

,ID,EFFECTIVE_FROM,EFFECTIVE_TO,Saved_by_teammate_per_round_both,Saved_by_teammate_per_round_CT,Saved_by_teammate_per_round_T,Traded_deaths_per_round_both,Traded_deaths_per_round_CT,Traded_deaths_per_round_T,Traded_deaths_percentage_both,Traded_deaths_percentage_CT,Traded_deaths_percentage_T,Opening_deaths_traded_percentage_both,Opening_deaths_traded_percentage_CT,Opening_deaths_traded_percentage_T,Assists_per_round_both,Assists_per_round_CT,Assists_per_round_T,Support_rounds_both,Support_rounds_CT,Support_rounds_T
0,10394,2025-07-15,2025-10-15,0.12,0.11,0.13,0.17,0.15,0.18,25.4,25.200001,25.5,37.299999,43.799999,33.700001,0.21,0.24,0.17,20.6,22.1,19.1


In [57]:
make_dataframe(soup=soup, name="entrying")

,ID,EFFECTIVE_FROM,EFFECTIVE_TO,Saved_by_teammate_per_round_both,Saved_by_teammate_per_round_CT,Saved_by_teammate_per_round_T,Traded_deaths_per_round_both,Traded_deaths_per_round_CT,Traded_deaths_per_round_T,Traded_deaths_percentage_both,Traded_deaths_percentage_CT,Traded_deaths_percentage_T,Opening_deaths_traded_percentage_both,Opening_deaths_traded_percentage_CT,Opening_deaths_traded_percentage_T,Assists_per_round_both,Assists_per_round_CT,Assists_per_round_T,Support_rounds_both,Support_rounds_CT,Support_rounds_T
0,10394,2025-08-04,2025-11-04,0.12,0.11,0.12,0.16,0.14,0.18,24.8%,24.2%,25.3%,38.7%,37.5%,39.5%,0.20,0.24,0.17,19.8%,21.4%,18.2%


## Trading

In [65]:
xxx1 = soup.find_all("div", {"class": "role-stats-section role-trading"})

In [66]:
trading = xxx1[0].find("div", {"class": "role-stats-margin"})

In [67]:
columns = [
    "ID",
    "EFFECTIVE_FROM",
    "EFFECTIVE_TO",
    "Saved_by_teammate_per_round_both",
    "Saved_by_teammate_per_round_CT",
    "Saved_by_teammate_per_round_T",
    "Trade_kills_per_round_both",
    "Trade_kills_per_round_CT",
    "Trade_kills_per_round_T",
    "Trade_kills_percentage_both",
    "Trade_kills_percentage_CT",
    "Trade_kills_percentage_T",
    "Assisted_kills_percentage_both",
    "Assisted_kills_percentage_CT",
    "Assisted_kills_percentage_T",
    "Damage_per_round_both",
    "Damage_per_round_CT",
    "Damage_per_round_T",
    ]

In [68]:
data = trading.find_all("div", {"class": "role-stats-data"})

In [69]:
data_column = np.zeros((1, len(data)+3))
data_column = data_column.astype(str)
for i in range(len(data)):
    data_column[0, i+3] = extract_string_between_chars(str(data[i]), ">", "<")
data_column[0, 0] = str(id)
data_column[0, 1] = date_3months
data_column[0, 2] = end_date

In [70]:
result_trading = pd.DataFrame(columns=columns, data=data_column)

In [71]:
percent_columns = [
    "Trade_kills_percentage_both",
    "Trade_kills_percentage_CT",
    "Trade_kills_percentage_T",
    "Assisted_kills_percentage_both",
    "Assisted_kills_percentage_CT",
    "Assisted_kills_percentage_T",
]

In [72]:
for column in percent_columns:
    result_trading[column] = result_trading[column].str.split("%")[0][0]

In [73]:
columns_types = {
    "ID": np.int32,
    "EFFECTIVE_FROM": np.datetime64,
    "EFFECTIVE_TO": np.datetime64,
    "Saved_by_teammate_per_round_both": np.float32,
    "Saved_by_teammate_per_round_CT": np.float32,
    "Saved_by_teammate_per_round_T": np.float32,
    "Trade_kills_per_round_both": np.float32,
    "Trade_kills_per_round_CT": np.float32,
    "Trade_kills_per_round_T": np.float32,
    "Trade_kills_percentage_both": np.float32,
    "Trade_kills_percentage_CT": np.float32,
    "Trade_kills_percentage_T": np.float32,
    "Assisted_kills_percentage_both": np.float32,
    "Assisted_kills_percentage_CT": np.float32,
    "Assisted_kills_percentage_T": np.float32,
    "Damage_per_round_both": np.float32,
    "Damage_per_round_CT": np.float32,
    "Damage_per_round_T": np.float32,
}

In [74]:
result_trading = make_types(columns_types, result_trading)

In [75]:
result_trading

,ID,EFFECTIVE_FROM,EFFECTIVE_TO,Saved_by_teammate_per_round_both,Saved_by_teammate_per_round_CT,Saved_by_teammate_per_round_T,Trade_kills_per_round_both,Trade_kills_per_round_CT,Trade_kills_per_round_T,Trade_kills_percentage_both,Trade_kills_percentage_CT,Trade_kills_percentage_T,Assisted_kills_percentage_both,Assisted_kills_percentage_CT,Assisted_kills_percentage_T,Damage_per_round_both,Damage_per_round_CT,Damage_per_round_T
0,10394,2025-07-15,2025-10-15,0.14,0.13,0.15,0.18,0.18,0.19,24.799999,21.6,29.0,26.5,28.6,23.9,110.0,109.0,111.0


## Opening

In [76]:
def make_dataframe(soup, name: str) -> pd.DataFrame:
    section = soup.find_all("div", {"class": f"role-stats-section role-{name}"})
    stats = section[0].find("div", {"class": "role-stats-margin"})
    data = stats.find_all("div", {"class": "role-stats-data"})
    title_stats = stats.find_all("div", {"class": "role-stats-title"})

    cols = []
    cols.append("ID")
    cols.append("EFFECTIVE_FROM")
    cols.append("EFFECTIVE_TO")

    data_column = np.zeros((1, len(data)+3))
    data_column = data_column.astype(str)
    for i in range(len(data)):
        data_column[0, i+3] = data[i].text
        if (i % 3) == 0:
            cols.append(title_stats[i].text.replace(' ', '_') + "_both")
        elif (i % 3) == 1:
            cols.append(title_stats[i].text.replace(' ', '_') + "_CT")
        elif (i % 3) == 2:
            cols.append(title_stats[i].text.replace(' ', '_') + "_T")
    data_column[0, 0] = str(id)
    data_column[0, 1] = date_3months
    data_column[0, 2] = end_date

    return pd.DataFrame(columns=cols, data=data_column)

In [77]:
result_opening = make_dataframe(soup=soup, name="opening")

In [78]:
result_opening

,ID,EFFECTIVE_FROM,EFFECTIVE_TO,Opening_kills_per_round_both,Opening_kills_per_round_CT,Opening_kills_per_round_T,Opening_deaths_per_round_both,Opening_deaths_per_round_CT,Opening_deaths_per_round_T,Opening_attempts_both,Opening_attempts_CT,Opening_attempts_T,Opening_success_both,Opening_success_CT,Opening_success_T,Win%_after_opening_kill_both,Win%_after_opening_kill_CT,Win%_after_opening_kill_T,Attacks_per_round_both,Attacks_per_round_CT,Attacks_per_round_T
0,10394,2025-07-15,2025-10-15,0.09,0.11,0.07,0.09,0.06,0.11,18.0%,17.8%,18.3%,51.1%,64.7%,37.7%,76.4%,73.9%,80.8%,2.89,3.34,2.43


In [79]:
columns_types = {
    "ID": np.int32,
    "EFFECTIVE_FROM": np.datetime64,
    "EFFECTIVE_TO": np.datetime64,
    "Opening_kills_per_round_both": np.float32,
    "Opening_kills_per_round_CT": np.float32,
    "Opening_kills_per_round_T": np.float32,
    "Opening_deaths_per_round_both": np.float32,
    "Opening_deaths_per_round_CT": np.float32,
    "Opening_deaths_per_round_T": np.float32,
    "Opening_attempts_both": np.float32,
    "Opening_attempts_CT": np.float32,
    "Opening_attempts_T": np.float32,
    "Opening_success_both": np.float32,
    "Opening_success_CT": np.float32,
    "Opening_success_T": np.float32,
    "Win%_after_opening_kill_both": np.float32,
    "Win%_after_opening_kill_CT": np.float32,
    "Win%_after_opening_kill_T": np.float32,
    "Attacks_per_round_both": np.float32,
    "Attacks_per_round_CT": np.float32,
    "Attacks_per_round_T": np.float32,
}

In [80]:
result_opening = make_types(columns_types, result_opening)

In [81]:
result_opening

,ID,EFFECTIVE_FROM,EFFECTIVE_TO,Opening_kills_per_round_both,Opening_kills_per_round_CT,Opening_kills_per_round_T,Opening_deaths_per_round_both,Opening_deaths_per_round_CT,Opening_deaths_per_round_T,Opening_attempts_both,Opening_attempts_CT,Opening_attempts_T,Opening_success_both,Opening_success_CT,Opening_success_T,Win%_after_opening_kill_both,Win%_after_opening_kill_CT,Win%_after_opening_kill_T,Attacks_per_round_both,Attacks_per_round_CT,Attacks_per_round_T
0,10394,2025-07-15,2025-10-15,0.09,0.11,0.07,0.09,0.06,0.11,18.0,17.799999,18.299999,51.099998,64.699997,37.700001,76.400002,73.900002,80.800003,2.89,3.34,2.43


## Clutching

In [82]:
result_clutching = make_dataframe(soup=soup, name="clutching")

In [83]:
result_clutching

,ID,EFFECTIVE_FROM,EFFECTIVE_TO,Clutch_points_per_round_both,Clutch_points_per_round_CT,Clutch_points_per_round_T,Last_alive_percentage_both,Last_alive_percentage_CT,Last_alive_percentage_T,1on1_win_percentage_both,1on1_win_percentage_CT,1on1_win_percentage_T,Time_alive_per_round_both,Time_alive_per_round_CT,Time_alive_per_round_T,Saves_per_round_loss_both,Saves_per_round_loss_CT,Saves_per_round_loss_T
0,10394,2025-07-15,2025-10-15,0.03,0.02,0.04,10.5%,8.1%,13.0%,50.0%,33.3%,63.6%,1m 8s,1m 12s,1m 5s,10.4%,18.7%,2.9%


In [84]:
columns_types = {
    "ID": np.int32,
    "EFFECTIVE_FROM": np.datetime64,
    "EFFECTIVE_TO": np.datetime64,
    "Clutch_points_per_round_both": np.float32,
    "Clutch_points_per_round_CT": np.float32,
    "Clutch_points_per_round_T": np.float32,
    "Last_alive_percentage_both": np.float32,
    "Last_alive_percentage_CT": np.float32,
    "Last_alive_percentage_T": np.float32,
    "1on1_win_percentage_both": np.float32,
    "1on1_win_percentage_CT": np.float32,
    "1on1_win_percentage_T": np.float32,
    "Time_alive_per_round_both": np.timedelta64,
    "Time_alive_per_round_CT": np.timedelta64,
    "Time_alive_per_round_T": np.timedelta64,
    "Saves_per_round_loss_both": np.float32,
    "Saves_per_round_loss_CT": np.float32,
    "Saves_per_round_loss_T": np.float32,
}

In [85]:
result_clutching = make_types(columns_types, result_clutching)

In [86]:
result_clutching

,ID,EFFECTIVE_FROM,EFFECTIVE_TO,Clutch_points_per_round_both,Clutch_points_per_round_CT,Clutch_points_per_round_T,Last_alive_percentage_both,Last_alive_percentage_CT,Last_alive_percentage_T,1on1_win_percentage_both,1on1_win_percentage_CT,1on1_win_percentage_T,Time_alive_per_round_both,Time_alive_per_round_CT,Time_alive_per_round_T,Saves_per_round_loss_both,Saves_per_round_loss_CT,Saves_per_round_loss_T
0,10394,2025-07-15,2025-10-15,0.03,0.02,0.04,10.5,8.1,13.0,50.0,33.299999,63.599998,0 days 00:01:08,0 days 00:01:12,0 days 00:01:05,10.4,18.700001,2.9


## Sniping

In [87]:
result_sniping = make_dataframe(soup=soup, name="sniping")

In [88]:
result_sniping

,ID,EFFECTIVE_FROM,EFFECTIVE_TO,Sniper_kills_per_round_both,Sniper_kills_per_round_CT,Sniper_kills_per_round_T,Sniper_kills_percentage_both,Sniper_kills_percentage_CT,Sniper_kills_percentage_T,Rounds_with_sniper_kills_percentage_both,Rounds_with_sniper_kills_percentage_CT,Rounds_with_sniper_kills_percentage_T,Sniper_multi-kill_rounds_both,Sniper_multi-kill_rounds_CT,Sniper_multi-kill_rounds_T,Sniper_opening_kills_per_round_both,Sniper_opening_kills_per_round_CT,Sniper_opening_kills_per_round_T
0,10394,2025-07-15,2025-10-15,0.00,0.00,0.01,0.4%,0.0%,0.8%,0.4%,-,0.8%,0.0007,0.0,0.001,0.0,0.0,0.0


In [89]:
columns_types = {
    "ID": np.int32,
    "EFFECTIVE_FROM": np.datetime64,
    "EFFECTIVE_TO": np.datetime64,
    "Sniper_kills_per_round_both": np.float32,
    "Sniper_kills_per_round_CT": np.float32,
    "Sniper_kills_per_round_T": np.float32,
    "Sniper_kills_percentage_both": np.float32,
    "Sniper_kills_percentage_CT": np.float32,
    "Sniper_kills_percentage_T": np.float32,
    "Rounds_with_sniper_kills_percentage_both": np.float32,
    "Rounds_with_sniper_kills_percentage_CT": np.float32,
    "Rounds_with_sniper_kills_percentage_T": np.float32,
    "Sniper_multi-kill_rounds_both": np.float32,
    "Sniper_multi-kill_rounds_CT": np.float32,
    "Sniper_multi-kill_rounds_T": np.float32,
    "Sniper_opening_kills_per_round_both": np.float32,
    "Sniper_opening_kills_per_round_CT": np.float32,
    "Sniper_opening_kills_per_round_T": np.float32,
}

In [90]:
result_sniping = make_types(columns_types, result_sniping)

In [91]:
result_sniping

,ID,EFFECTIVE_FROM,EFFECTIVE_TO,Sniper_kills_per_round_both,Sniper_kills_per_round_CT,Sniper_kills_per_round_T,Sniper_kills_percentage_both,Sniper_kills_percentage_CT,Sniper_kills_percentage_T,Rounds_with_sniper_kills_percentage_both,Rounds_with_sniper_kills_percentage_CT,Rounds_with_sniper_kills_percentage_T,Sniper_multi-kill_rounds_both,Sniper_multi-kill_rounds_CT,Sniper_multi-kill_rounds_T,Sniper_opening_kills_per_round_both,Sniper_opening_kills_per_round_CT,Sniper_opening_kills_per_round_T
0,10394,2025-07-15,2025-10-15,0.0,0.0,0.01,0.4,0.0,0.8,0.4,0.0,0.8,0.0007,0.0,0.001,0.0,0.0,0.0


## Utility

In [92]:
result_utility = make_dataframe(soup=soup, name="utility")

In [93]:
result_utility

,ID,EFFECTIVE_FROM,EFFECTIVE_TO,Utility_damage_per_round_both,Utility_damage_per_round_CT,Utility_damage_per_round_T,Utility_kills_per_100_rounds_both,Utility_kills_per_100_rounds_CT,Utility_kills_per_100_rounds_T,Flashes_thrown_per_round_both,Flashes_thrown_per_round_CT,Flashes_thrown_per_round_T,Flash_assists_per_round_both,Flash_assists_per_round_CT,Flash_assists_per_round_T,Time_opponent_flashed_per_round_both,Time_opponent_flashed_per_round_CT,Time_opponent_flashed_per_round_T
0,10394,2025-07-15,2025-10-15,5.37,7.11,3.59,0.59,0.65,0.53,0.73,0.64,0.84,0.04,0.03,0.05,2.18,1.98,2.33


In [94]:
columns_types = {
    "ID": np.int32,
    "EFFECTIVE_FROM": np.datetime64,
    "EFFECTIVE_TO": np.datetime64,
    "Utility_damage_per_round_both": np.float32,
    "Utility_damage_per_round_CT": np.float32,
    "Utility_damage_per_round_T": np.float32,
    "Utility_kills_per_100_rounds_both": np.float32,
    "Utility_kills_per_100_rounds_CT": np.float32,
    "Utility_kills_per_100_rounds_T": np.float32,
    "Flashes_thrown_per_round_both": np.float32,
    "Flashes_thrown_per_round_CT": np.float32,
    "Flashes_thrown_per_round_T": np.float32,
    "Flash_assists_per_round_both": np.float32,
    "Flash_assists_per_round_CT": np.float32,
    "Flash_assists_per_round_T": np.float32,
    "Time_opponent_flashed_per_round_both": np.float32,
    "Time_opponent_flashed_per_round_CT": np.float32,
    "Time_opponent_flashed_per_round_T": np.float32,
}

In [95]:
result_utility = make_types(columns_types, result_utility)

In [96]:
result_utility

,ID,EFFECTIVE_FROM,EFFECTIVE_TO,Utility_damage_per_round_both,Utility_damage_per_round_CT,Utility_damage_per_round_T,Utility_kills_per_100_rounds_both,Utility_kills_per_100_rounds_CT,Utility_kills_per_100_rounds_T,Flashes_thrown_per_round_both,Flashes_thrown_per_round_CT,Flashes_thrown_per_round_T,Flash_assists_per_round_both,Flash_assists_per_round_CT,Flash_assists_per_round_T,Time_opponent_flashed_per_round_both,Time_opponent_flashed_per_round_CT,Time_opponent_flashed_per_round_T
0,10394,2025-07-15,2025-10-15,5.37,7.11,3.59,0.59,0.65,0.53,0.73,0.64,0.84,0.04,0.03,0.05,2.18,1.98,2.33


## Main statistics

In [97]:
xxx1 = soup.find_all("div", {"class": "statistics"})

In [98]:
main_statistics = xxx1[0].find_all("div", {"class": "stats-row"})

In [99]:
data_column = np.zeros((1, len(main_statistics)+3))
data_column = data_column.astype(str)

cols = []
cols.append("ID")
cols.append("EFFECTIVE_FROM")
cols.append("EFFECTIVE_TO")

for i in range(len(main_statistics)):
    full_text = main_statistics[i].get_text("|")
    cols.append((full_text.split("|")[0]
                 .replace(" ", "_")
                 .replace("_/_Round", "_per_round")
                 .replace("_/_round", "_per_round")))
    data_column[0, i+3] = full_text.split("|")[1]
data_column[0, 0] = str(id)
data_column[0, 1] = date_3months
data_column[0, 2] = end_date

In [100]:
result_statistics = pd.DataFrame(columns=cols, data=data_column)

In [101]:
result_statistics

,ID,EFFECTIVE_FROM,EFFECTIVE_TO,Total_kills,Headshot_%,Total_deaths,K/D_Ratio,Damage_per_round,Grenade_dmg_per_round,Maps_played,Rounds_played,Kills_per_round,Assists_per_round,Deaths_per_round,Saved_by_teammate_per_round,Saved_teammates_per_round,Impact_rating
0,10394,2025-07-15,2025-10-15,1130,59.4%,994,1.14,82.1,5.4,69,1519,0.74,0.21,0.65,0.12,0.14,1.14


In [102]:
columns_types = {
    "ID": np.int32,
    "EFFECTIVE_FROM": np.datetime64,
    "EFFECTIVE_TO": np.datetime64,
    "Total_kills": np.int64,
    "Headshot_%": np.float32,
    "Total_deaths": np.float32,
    "K/D_Ratio": np.float32,
    "Damage_per_round": np.float32,
    "Grenade_dmg_per_round": np.float32,
    "Maps_played": np.float32,
    "Rounds_played": np.float32,
    "Kills_per_round": np.float32,
    "Assists_per_round": np.float32,
    "Deaths_per_round": np.float32,
    "Saved_by_teammate_per_round": np.float32,
    "Saved_teammates_per_round": np.float32,
    "Impact_rating": np.float32,
}

In [103]:
result_statistics = make_types(columns_types, result_statistics)

In [104]:
result_statistics

,ID,EFFECTIVE_FROM,EFFECTIVE_TO,Total_kills,Headshot_%,Total_deaths,K/D_Ratio,Damage_per_round,Grenade_dmg_per_round,Maps_played,Rounds_played,Kills_per_round,Assists_per_round,Deaths_per_round,Saved_by_teammate_per_round,Saved_teammates_per_round,Impact_rating
0,10394,2025-07-15,2025-10-15,1130,59.400002,994.0,1.14,82.099998,5.4,69.0,1519.0,0.74,0.21,0.65,0.12,0.14,1.14


## Player data

In [114]:
player

{'name': "Russel 'Twistzz' Van Dulken",
 'player-id': 10394,
 'url': 'https://www.hltv.org/player/10394/twistzz'}

In [115]:
kk = player["name"].split("'")

In [116]:
kkk = kk

In [117]:
ll = kkk.pop(1)

In [118]:
ll

'Twistzz'

In [119]:
kkk

['Russel ', ' Van Dulken']

In [147]:
name = ' '.join(kkk[0].split(" ")[:-1])

In [148]:
surname = ' '.join(kkk[1].split(" ")[1:])

In [126]:
name

"Russel 'Twistzz' Van Dulken"

In [127]:
nickname

'Twistzz'

In [129]:
id

10394

In [134]:
url = player["url"]

In [135]:
url

'https://www.hltv.org/player/10394/twistzz'

In [136]:
player

{'name': "Russel 'Twistzz' Van Dulken",
 'player-id': 10394,
 'url': 'https://www.hltv.org/player/10394/twistzz'}

In [138]:
pd.DataFrame(data=[player])

,name,player-id,url
0,Russel 'Twistzz' Van Dulken,10394,https://www.hltv.org/player/10394/twistzz


In [149]:
data_to_df = {
    "id": id,
    "name": name,
    "surname": surname,
    "nickname": nickname,
    "team": team.values[0],
    "url": url,
}

In [153]:
pd.DataFrame(data=data_to_df, index=[0])

,id,name,surname,nickname,team,url
0,10394,Russel,Van Dulken,Twistzz,FaZe,https://www.hltv.org/player/10394/twistzz
